## Prepare example SHACL profiles from JSON API

In [1]:
import requests
import pprint
import pandas as pd

In [2]:
profile_id = 5 #176 #198

In [ ]:
profile_classes_url = 'https://ontome.net/api/classes-profile.json?lang=en&available-in-profile=' + str(profile_id)

try:
    response = requests.get(profile_classes_url)
    json_classes = response.json()
    print(len(json_classes))
except Exception as e:
    print(e)


In [23]:
### API output

# print(json_classes)

In [ ]:
l = []

for cla in json_classes:
    l.append('@prefix ' + cla["namespacePrefix"] + ': <' + cla["namespaceURI"] + '>.')

classes_prefixes = list(set(l))    #list(pd.unique(pd.DataFrame(l)[0]))

pprint.pprint(classes_prefixes)

In [ ]:
### Classes and their elements

# classes list
lc = []
for cla in json_classes:
    lc.append([cla['namespacePrefix'] , cla["namespaceURI"], cla["classIdentifierInNamespace"], 
               cla['classID'], cla["classLabel"], cla["entityBasicType"], cla["ancestorClasses"]])

df_lc = pd.DataFrame(lc)
df_lc.columns = ['pref', 'ns_uri', 'uri', 'id', 'label', 'type', 'ancestors']
df_lc

In [ ]:
for index, row in df_lc.iterrows():
    print(row['uri'])

In [ ]:

profile_properties_url = 'https://ontome.net/api/properties-profile.json?lang=en&available-in-profile=' + str(profile_id)

try:
    response = requests.get(profile_properties_url)
    json_properties = response.json()
    print(len(json_properties))
except Exception as e:
    print(e)


In [ ]:
l = []

for prop in json_properties:
    l.append('@prefix ' + prop["namespacePrefix"] + ': <' + prop["namespaceURI"] + '>.')


properties_prefixes = list(set(l)) #list(pd.unique(pd.DataFrame(l)[0]))

pprint.pprint(properties_prefixes)

In [ ]:
### properties and their elements

# properties list
lp = []
for prop in json_properties:
    lp.append([ prop["propertyDomain"], prop["domainInstancesMinQuantifier"], prop["domainInstancesMaxQuantifier"], 
               prop['namespacePrefix'], prop['namespaceURI'], 
               prop["propertyIdentifierInNamespace"], prop['propertyLabel'], 
               prop["propertyRange"], prop["rangeInstancesMinQuantifier"],prop["rangeInstancesMaxQuantifier"]])

df_lp = pd.DataFrame(lp)
df_lp.columns = ['id_s', 'min_s', 'max_s', 'ns_prefix', 'ns_uri', 'uri', 'label', 'id_t', 'min_t', 'max_t']
df_lp

In [ ]:
## get domain classes
 
df_ms = df_lp.merge(df_lc[['pref', 'uri', 'id']], left_on='id_s', right_on='id', suffixes=['_ps', '_so'] )
df_ms

In [ ]:
## get range classes

df_mt = df_ms.merge(df_lc[['pref', 'uri', 'id']], left_on='id_t', right_on='id', suffixes=['_pt', '_ta'] )
df_mt

In [ ]:
### new properties list

a = df_mt[['pref_pt', 'uri_so', 'min_s', 'max_s', 'ns_prefix', 'ns_uri', 'uri_ps', 'label', 'pref_ta', 'uri', 'min_t', 'max_t']].to_records()


prop_l = list(a)



pprint.pprint(prop_l)

### Prepare the output ttl file

In [ ]:
### Namespaces list
ns_list = list(set(classes_prefixes + properties_prefixes))
ns_list_txt = '\n'.join(ns_list)
print(ns_list_txt)

In [ ]:
print(lc[:1])

In [ ]:
pprint.pprint(prop_l)

In [ ]:
print(prop_l[2][4])

In [ ]:
### Write a file with the corresponding shacl profile

shacl_prof = """### SHACL OntoME Profile {} \n\n

@prefix sdh-shacl: <https://sdhss.org/shacl/profiles/>. 
@prefix sh: <http://www.w3.org/ns/shacl#> .  
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: 	<http://www.w3.org/2000/01/rdf-schema#> .  
@prefix sdh-short: <https://sdhss.org/ontology/shortcuts/>.
""".format(profile_id) + ns_list_txt

## for each class
for c in lc: #[1:2]:
    
    if c[5] == 9:
        min_count_label_comment = ''
        temporal_properties_standard_time = """
        sh:property [
        sh:path sdh-short:P1 ;
        sh:name "Standard Date-Time";
        sh:datatype xsd:string;
        sh:order 3;
        ];
        """
    else:
        min_count_label_comment = 'sh:minCount 1;'
        temporal_properties_standard_time = ''
    

    if c[0]+':'+ c[2] == 'crm:E21':
        print('yes')
        person_birth_date = """
    sh:property [
        sh:path sdh-short:P2 ;
        sh:name "has birth date";
        sh:datatype xsd:string;
        sh:order 3;
        ];
        """
    else:
        person_birth_date = ''   

    shacl_prof += """ \n
sdh-shacl:{0}_{1}_Shape a sh:NodeShape ;
    sh:targetClass {0}:{1} ;
    sh:name "{2}" ;

    sh:property [
        sh:path rdfs:label ;
        sh:name "Label";
        sh:datatype xsd:string;
        sh:order 1;
        {3}
        sh:maxCount 1;
        ];

    sh:property [
        sh:path rdfs:comment ;
        sh:name "Description";
        sh:datatype rdf:HTML;
        sh:order 2;
        {3}
        ];   
    {5}
    {4}    
    """.format(c[0], c[2], c[4], min_count_label_comment,
               temporal_properties_standard_time, person_birth_date)

    ## outgoing properties
    for p in prop_l:
        
        if p[12] == 1:
            max_count = 'sh:maxCount 1 ;'
        else:
            max_count = ''    

        if (p[1] == c[0] and p[2] == c[2]) :
            
            pl = """    
sh:property [
        sh:path {0}:{1} ;
        sh:name "{2}";
        sh:class {3}:{4} ;
        {5}
        ];
        """.format(p[5], p[7], p[8], p[9], p[10], max_count)


            shacl_prof += pl

    ## incoming properties
    for p in prop_l:

        
        if (p[9] == c[0] and p[10] == c[2] \
            ### we choose here the classes with inverse properties shown
            and p[9]+':'+ p[10] in ['crm:E21']) :
            if p[4] == 1:
                max_count_i = 'sh:maxCount 1 ;'
            else:
                max_count_i = '' 

            pl = """
    sh:property [
        sh:name "Inverse of: {5}" ;
        {4}
        sh:path [ 
            sh:inversePath {0}:{1};
            ### None of both is not valid in SHACL
            # sh:class — sh:targetClass {2}:{3};
            ] ;
        ]; """.format(p[5], p[7], p[1], p[2], max_count_i, p[8])

            shacl_prof += pl

    shacl_prof += '.'


# print(shacl_prof)

## write file            
file_address = "shacl-profile-{}.ttl".format(profile_id)
with open(file_address, "w") as text_file:
    text_file.write(shacl_prof)


'pref_sub', 'uri_sub', 'min_sub', 'max_sub', 'ns_prefix', 'ns_pp', 'uri_pp', 'pp_label', 'pref_ob', 'uri_ob', 'min_ob', 'max_ob'

In [ ]:
pprint.pprint(prop_l)

### Example of inverse property

In [ ]:
### Write a file with the corresponding shacl profile

shacl_prof = """### SHACL OntoME Profile {} \n\n

@prefix sdh-shacl: <https://sdhss.org/shacl/profiles/>. 
@prefix sh: <http://www.w3.org/ns/shacl#> .  
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: 	<http://www.w3.org/2000/01/rdf-schema#> .  
@prefix sdh-short: <https://sdhss.org/ontology/shortcuts/P1>.
""".format(profile_id) + ns_list_txt

## for each class
for c in lc: #[1:2]:
    
    if c[5] == 9:
        min_count_label_comment = ''
        temporal_properties_standard_time = """
        sh:property [
        sh:path sdh-short:P1 ;
        sh:name "Standard Date-Time";
        sh:datatype xsd:string;
        sh:order 3;
        ];
        """
    else:
        min_count_label_comment = 'sh:minCount 1;'
        temporal_properties_standard_time = ''
    

    if c[0]+':'+ c[2] == 'crm:E21':
        print('yes')
        person_birth_date = """
    sh:property [
        sh:path sdh-short:P2 ;
        sh:name "has birth date";
        sh:datatype xsd:string;
        sh:order 3;
        ];
        """
    else:
        person_birth_date = ''   

    shacl_prof += """ \n
sdh-shacl:{0}_{1}_Shape a sh:NodeShape ;
    sh:targetClass {0}:{1} ;
    sh:name "{2}" ;

    sh:property [
        sh:path rdfs:label ;
        sh:name "Label";
        sh:datatype xsd:string;
        sh:order 1;
        {3}
        sh:maxCount 1;
        ];

    sh:property [
        sh:path rdfs:comment ;
        sh:name "Description";
        sh:datatype rdf:HTML;
        sh:order 2;
        {3}
        ];   
    {5}
    {4}    
    """.format(c[0], c[2], c[4], min_count_label_comment,
               temporal_properties_standard_time, person_birth_date)

    ## outgoing properties
    for p in prop_l:
        
        if p[12] == 1:
            max_count = 'sh:maxCount 1 ;'
        else:
            max_count = ''    

        if (p[1] == c[0] and p[2] == c[2]) :
            
            pl = """    
sh:property [
        sh:path {0}:{1} ;
        sh:name "{2}";
        sh:class {3}:{4} ;
        {5}
        ];
        """.format(p[5], p[7], p[8], p[9], p[10], max_count)


            shacl_prof += pl

    ## incoming properties
    for p in prop_l:

        
        if (p[9] == c[0] and p[10] == c[2] \
            ### we choose here the classes with inverse properties shown
            and p[9]+':'+ p[10] in ['crm:E21']) :
            if p[4] == 1:
                max_count_i = 'sh:maxCount 1 ;'
            else:
                max_count_i = '' 

            pl = """
    sh:property [
        sh:name "Inverse of: {5}" ;
        {4}
        sh:path [ 
            sh:inversePath {0}:{1};
            ### None of both is not valid in SHACL
            # sh:class — sh:targetClass {2}:{3};
            ] ;
        ]; """.format(p[5], p[7], p[1], p[2], max_count_i, p[8])

            shacl_prof += pl

    shacl_prof += '.'


# print(shacl_prof)

## write file            
file_address = "shacl-profile-{}.ttl".format(profile_id)
with open(file_address, "w") as text_file:
    text_file.write(shacl_prof)
